## Spatial analysis

first let's nstall dependencies for spatial analysis

In [2]:
! pip install geopandas
! pip install geocube

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


Import all needed python libraries for data processing

In [25]:
import geopandas as gpd
import pandas as pd
from geocube.api.core import make_geocube
import numpy as np
import xarray as xr
from shapely.geometry import Point

## Data Collection/Preprocessing

Input data is located under data folder and splitted by year(the source data is downloaded from eurostat).

We first read slovakia boundary shapefile to get vectorial data and  join it with csv file containing census data

In [4]:
euro_vector_2006 = gpd.read_file('data/2006/Grid_ETRS89_LAEA_1K_ref_GEOSTAT_2006.shp')

In [8]:
euro_census_2006 = pd.read_csv("data/2006/GEOSTAT_grid_EU_POP_2006_1K_V1_1_1.csv", sep=';')


The output contains 3 columns identifying : the grid id , population count and the geometry (polygon vector)

In [13]:
euro_global_2006 = euro_census_2006.merge(euro_vector_2006, left_on="GRD_ID", right_on="GRD_INSPIR")[["GRD_ID", "POP_TOT", "geometry"]]
euro_global_2006

,GRD_ID,POP_TOT,geometry
0,1kmN5142E2862,2,"POLYGON ((2862000.000 5142000.000, 2862000.000..."
1,1kmN5141E2862,13,"POLYGON ((2862000.000 5141000.000, 2862000.000..."
2,1kmN5141E2864,211,"POLYGON ((2864000.000 5141000.000, 2864000.000..."
3,1kmN5140E2862,1,"POLYGON ((2862000.000 5140000.000, 2862000.000..."
4,1kmN5139E2876,33,"POLYGON ((2876000.000 5139000.000, 2876000.000..."
...,...,...,...
1946456,1kmN4323E4937,1004,"POLYGON ((4937000.000 4323000.000, 4937000.000..."
1946457,1kmN4276E4938,1921,"POLYGON ((4938000.000 4276000.000, 4938000.000..."
1946458,1kmN4277E4938,1328,"POLYGON ((4938000.000 4277000.000, 4938000.000..."
1946459,1kmN4323E4938,1042,"POLYGON ((4938000.000 4323000.000, 4938000.000..."


## Rasterization

Using make_geocube library we convert the vector data (polygon) to grid data with 1 km square pixel resolution
The output is a xarray type 

In [14]:
census_raster_2006 = make_geocube(vector_data=euro_global_2006, measurements=['POP_TOT'], resolution=(1000, -1000))
census_raster_2006

<xarray.Dataset>
Dimensions:      (y: 4472, x: 5012)
Coordinates:
  * y            (y) float64 9.425e+05 9.435e+05 ... 5.412e+06 5.414e+06
  * x            (x) float64 5.956e+06 5.954e+06 ... 9.455e+05 9.445e+05
    spatial_ref  int64 0
Data variables:
    POP_TOT      (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan

Similar preprocessing for 2021 spatial data :
We read slovakia boundary geo-package file to get vectorial data using geopandas library

In [15]:
euro_vector_2021 = gpd.read_file("data/2021/ESTAT_Census_2021_V1-0.gpkg")

Renaming the geodataframe columns

In [21]:
euro_vector_2021.columns = ['GRD_ID', 'POP_TOT', 'geometry']
euro_vector_2021

,GRD_ID,POP_TOT,geometry
0,CRS3035RES1000mN1000000E1964000,0.0,"POLYGON ((1964000.000 1000000.000, 1965000.000..."
1,CRS3035RES1000mN1000000E1965000,0.0,"POLYGON ((1965000.000 1000000.000, 1966000.000..."
2,CRS3035RES1000mN1000000E1966000,118.0,"POLYGON ((1966000.000 1000000.000, 1967000.000..."
3,CRS3035RES1000mN1000000E1967000,4.0,"POLYGON ((1967000.000 1000000.000, 1968000.000..."
4,CRS3035RES1000mN1000000E1968000,0.0,"POLYGON ((1968000.000 1000000.000, 1969000.000..."
...,...,...,...
4688513,CRS3035RES1000mN999000E1984000,0.0,"POLYGON ((1984000.000 999000.000, 1985000.000 ..."
4688514,CRS3035RES1000mN999000E1985000,0.0,"POLYGON ((1985000.000 999000.000, 1986000.000 ..."
4688515,CRS3035RES1000mN999000E1986000,0.0,"POLYGON ((1986000.000 999000.000, 1987000.000 ..."
4688516,CRS3035RES1000mN999000E1987000,0.0,"POLYGON ((1987000.000 999000.000, 1988000.000 ..."


Rasterizing euro census Slovakia 2021 grid data with 1Km square pixel resolution

In [17]:
census_raster_2021 = make_geocube(vector_data=euro_vector_2021, measurements=['POP_TOT'], resolution=(1000, -1000))

Aligning the two rasters for dimension matching

In [18]:
# Align the two rasters along their dimensions
census_raster_2006, census_raster_2021 = xr.align(census_raster_2006, census_raster_2021, join='outer')


Substracting to get relative change

In [19]:
# Substracting to get relative change
result = np.abs(census_raster_2021 - census_raster_2006)
result = result.fillna(0)

In [22]:
result

<xarray.Dataset>
Dimensions:      (y: 4478, x: 5563)
Coordinates:
  * y            (y) float64 9.405e+05 9.415e+05 ... 5.416e+06 5.418e+06
  * x            (x) float64 9.425e+05 9.435e+05 ... 6.504e+06 6.504e+06
    spatial_ref  int64 0
Data variables:
    POP_TOT      (y, x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

top 5 biggest relative change

In [32]:
top5_change = sorted(result.POP_TOT.to_numpy().flatten(), reverse=True)[:5]
top5_change

[29508.0, 22600.0, 21626.0, 21530.0, 19891.0]

Filtering the result with top 5

In [35]:
result = result.where(result.POP_TOT >= top5_change[4])
result = result.fillna(0)
result

<xarray.Dataset>
Dimensions:      (y: 4478, x: 5563)
Coordinates:
  * y            (y) float64 9.405e+05 9.415e+05 ... 5.416e+06 5.418e+06
  * x            (x) float64 9.425e+05 9.435e+05 ... 6.504e+06 6.504e+06
    spatial_ref  int64 0
Data variables:
    POP_TOT      (y, x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

Create the following Columns:

Latitude and Longitude of centers of those 5 areas

Population for 2006

Population for 2021

Relative changes


In [59]:
pop_relative_change = result.POP_TOT.to_numpy()
x, y = np.meshgrid(result.x, result.y)

In [60]:
relative_pop = []
population_2006 = []
population_2021 = []
area_center = []

for i in range(pop_relative_change.shape[0]):
    for j in range(pop_relative_change.shape[1]):
        if pop_relative_change[i][j] > 0:
            relative_pop.append(pop_relative_change[i][j])
            area_center.append(Point(x[i][j], y[i][j]))
            population_2006.append(census_raster_2006.POP_TOT.to_numpy()[i][j])
            population_2021.append(census_raster_2021.POP_TOT.to_numpy()[i][j])


In [61]:
# Create a DataFrame with centroids and pixel values of relative difference
top_5_cells = pd.DataFrame({
    'area_center': area_center,
    'relative_population_difference': relative_pop,
    'population_2006': population_2006,
    'population_2021': population_2021
})

top_5_cells = top_5_cells.sort_values(by='relative_population_difference', ascending=False)

In [62]:
top_5_cells

,area_center,relative_population_difference,population_2006,population_2021
3,POINT (4481500 3618500),29508.0,240.0,29748.0
4,POINT (4484500 3621500),22600.0,9248.0,31848.0
2,POINT (3729500 3102500),21626.0,7542.0,29168.0
1,POINT (4534500 2087500),21530.0,11741.0,33271.0
0,POINT (5528500 1764500),19891.0,24545.0,4654.0


Saving top 5 result to CSV

In [65]:

top_5_cells.to_csv('data/output/top_5_cells.csv', index=False)

average and median population per 1 square km grid in Slovakia in 2021

In [66]:

mean = euro_vector_2021['POP_TOT'].mean()
median = euro_vector_2021['POP_TOT'].median()


create dataframe for average and median population per 1 square km grid

In [67]:
mean_median_df = pd.DataFrame([[mean, median]], columns=['mean', 'median'])
mean_median_df

,mean,median
0,97.731956,0.0


aving result to CSV

In [68]:

mean_median_df.to_csv('data/output/mean_median.csv', index=False)